In [1]:
pylab inline

# /bin/rm ~/.cache/fontconfig/*


SyntaxError: invalid syntax (<ipython-input-1-8ea824fbd00a>, line 1)

In [8]:
try:
    reload(dc)
    print  'dc reloaded'
except Exception:
    print 'dc not loaded yet'
    
WORDSIZE = 32
import dnnConnectivityAndMemoryAllocation as dc
import numpy as np
import matplotlib.pyplot as plt

    

dc reloaded


In [9]:
# Create memory
mainMemoryConfig = dc.MemoryConfiguration(numOfChannels=2,numOfBanksPerChannel=32,numOfPagesPerBank=8,sizeOfPage=4096)
mainMemory = dc.Memory(mainMemoryConfig)



In [10]:
# Create DNN
network = dc.Network()

In [11]:
#                                    X    Y    Z    Kx   Ky   Kz   stride
#network.addLayer('Input',          224, 224,    3                      ) #    3 
#network.addLayer('Convolutional',   55,  55,   96,   11,  11,    3,   4 ) #   96,
#network.addLayer('Convolutional',   27,  27,  256,    5,   5,   96,   2 ) #  256,
#network.addLayer('Convolutional',   13,  13,  384,    3,   3,  256,   2 ) #  384,
# network.addLayer('Convolutional',   13,  13,  384,    3,   3,  384,   1 ) #  384,
# network.addLayer('Fully Connected', 13,  13,  256,    3,   3,  384,   1 ) #  256,
# network.addLayer('Fully Connected',  1,   1, 4096,   13,  13,  256,   1 ) # 4096,
# network.addLayer('Fully Connected',  1,   1, 4096,    1,   1, 4096,   1 ) # 4096,
# network.addLayer('Fully Connected',  1,   1, 1024,    1,   1, 4096,   1 ) # 1024,

#network.addLayer('Input',           55,  55,    4,                      ) #   96,
network.addLayer('Input',          224, 224,    3                       ) #    3 
network.addLayer('Convolutional',   55,  55,    4,    8,   8,    3,   4 ) #   96,
network.addLayer('Convolutional',   27,  27,    8,    5,   5,    4,   2 ) #  256,
network.addLayer('Convolutional',   13,  13,    4,    3,   3,    8,   2 ) #  384,
#network.addLayer('Convolutional',   13,  13,    2,    3,   3,    4,   1 ) #  384,
#network.addLayer('Fully Connected', 13,  13,    6,    3,   3,    8,   1 ) #  256,
#network.addLayer('Fully Connected',  1,   1,    6,   13,  13,    6,   1 ) # 4096,
#network.addLayer('Fully Connected',  1,   1,    4,    1,   1,    6,   1 ) # 4096,
#network.addLayer('Fully Connected',  1,   1,    4,    1,   1,    4,   1 ) # 1024,



In [12]:
# Create processing engines and assign each layers cell to the array of PEs
#print '{0}:{1}:Assign PEs:'.format(__FILE__(), __LINE__())
network.assignPEs('linearAll')


Layer  0  left Kernel offset is  0 , top Kernel offset is  0
Layer  1  left Kernel offset is  0 , top Kernel offset is  0
Layer  2  left Kernel offset is  -1 , top Kernel offset is  -1
Layer  3  left Kernel offset is  0 , top Kernel offset is  0


In [13]:
# Create Manager and assign array memory type
#print '{0}:{1}:Assign Managers:'.format(__FILE__(), __LINE__())
mgrMemoryConfig = dc.MemoryConfiguration(numOfChannels=2,numOfBanksPerChannel=32,numOfPagesPerBank=32,sizeOfPage=4096)
network.assignManagers(mgrMemoryConfig)


In [14]:
for l in range(1, network.numberOfLayers):
      #print '{0}:{1}:Generate connections for layer {2}:'.format(__FILE__(), __LINE__(), l)
      network.Layers[l].generateConnections()



Updating Layer 1 connections for features in row :0
Updating Layer 1 connections for features in row :1
Updating Layer 1 connections for features in row :2
Updating Layer 1 connections for features in row :3
Updating Layer 1 connections for features in row :4
Updating Layer 1 connections for features in row :5
Updating Layer 1 connections for features in row :6
Updating Layer 1 connections for features in row :7
Updating Layer 1 connections for features in row :8
Updating Layer 1 connections for features in row :9
Updating Layer 1 connections for features in row :10
Updating Layer 1 connections for features in row :11
Updating Layer 1 connections for features in row :12
Updating Layer 1 connections for features in row :13
Updating Layer 1 connections for features in row :14
Updating Layer 1 connections for features in row :15
Updating Layer 1 connections for features in row :16
Updating Layer 1 connections for features in row :17
Updating Layer 1 connections for features in row :18
Upd

In [15]:
for l in range(1, network.numberOfLayers):
        #print '{0}:{1}:Determine ROI for layer {2}:'.format(__FILE__(), __LINE__(), l)
        network.peArray.findROIall(l)  



In [16]:
peMemoryAllocationOptions = dc.MemoryAllocationOptions( order             = ['c', 'b', 'p', 'w'],
                                                        initialChannel    =  0 , 
                                                        channelIncrement  =  1 , 
                                                        initialBank       =  0 , 
                                                        bankIncrement     =  2 , 
                                                        initialPage       =  0 , 
                                                        pageIncrement     =  2 , 
                                                        initialWord       =  0 , 
                                                        wordIncrement     =  1 , 
                                                        padWordRadix2     = 'Y')




In [ ]:
layerID = 1
mgrX = 1
mgrY = 0

In [ ]:
for l in range(1, network.numberOfLayers):
        for mgrY in range(network.managerArray.Y):
            for mgrX in range(network.managerArray.X):
                #print '{0}:{1}:Mem cpy ROI and memory allocation for layer {2}, manager {3},{4}'.format(__FILE__(), __LINE__(), l, mgrY, mgrX)
                rv = network.managerArray.manager[mgrY][mgrX].memCpyROI(layerID)
                network.managerArray.manager[mgrY][mgrX].allocateMemory(layerID, peMemoryAllocationOptions )
                network.managerArray.manager[mgrY][mgrX].createMemoryAllocationFile(layerID)

In [ ]:
#print '{0}:{1}:END:'.format(__FILE__(), __LINE__())